In [2]:
!git clone https://github.com/dlpbc/keras-kinetics-i3d.git
%cd keras-kinetics-i3d

/content/keras-kinetics-i3d


In [0]:
from google.colab import files

!mkdir train
%cd train

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'
          .format(name=fn, length=len(uploaded[fn])))

%cd ../

In [0]:
# !wget -P data https://github.com/deepmind/kinetics-i3d/raw/master/data/v_CricketShot_g04_c01_flow.npy
# !wget -P data https://github.com/deepmind/kinetics-i3d/raw/master/data/v_CricketShot_g04_c01_rgb.npy
# !ls data

In [0]:
import cv2
import glob
import numpy as np
import os
import random
import tensorflow as tf

from i3d_inception import Inception_Inflated3d
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.utils import to_categorical
from sklearn import preprocessing

In [0]:
class DataGenerator:
    
    def __init__(self, train_dir=None, test_dir=None, shuffle_train=False, validation_split=0.0, 
                 n_frames=16):
        self.n_frames = n_frames
        self.frame_h = 224
        self.frame_w = 224
        self.classes = ["kickflip", "heelflip"]
        self.n_classes = len(self.classes)
        self.label_encoder = preprocessing.LabelEncoder()
        self.label_encoder.fit(self.classes)
        
        if train_dir:
            pathname = os.path.join(train_dir, "*")
            filepaths = glob.glob(pathname)
            if shuffle_train:
                random.shuffle(filepaths)
            
            n_train = int((1.0-validation_split) * len(filepaths))
            self.train_filepaths = filepaths[:n_train]
            self.val_filepaths = filepaths[n_train:]
            
    def get_train_generator(self):
        while True:
            for train_filepath in self.train_filepaths:
                frames = self._get_frames(train_filepath)
                label = self._get_label(train_filepath)
                labels_encoded = self.label_encoder.transform([label])
                labels_one_hot = to_categorical(labels_encoded, 
                                                num_classes=self.n_classes).astype(int)

                yield np.array([frames]), labels_one_hot

    def _get_frames(self, filepath):
        video = cv2.VideoCapture(filepath)
        if not video.isOpened():
            raise FileNotFoundError("The input video path you provided is invalid.")
        
        frames = []
        while video.isOpened():
            grabbed, frame_bgr = video.read()
            if not grabbed:
                break
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        video.release()
        
        if len(frames) < self.n_frames:
            n_pad_frames = self.n_frames - len(frames)
            for _ in range(n_pad_frames):
                blank_frame = np.zeros((self.frame_h, self.frame_w, 3))
                frames.append(blank_frame)
        
        frames_processed = []
        current_ix = 0
        step_size = len(frames) / float(self.n_frames)
        for _ in range(self.n_frames):
            frame = cv2.resize(frames[int(current_ix)], (self.frame_h, self.frame_w))
            frames_processed.append(frame)
            current_ix += step_size
        
        return frames_processed

    def _get_label(self, filepath):
        file_name = filepath.split("/")[-1]
        return file_name.split("-")[0]

In [0]:
dg = DataGenerator(train_dir="train")
train_gen = dg.get_train_generator()

# ix = 0
# for frames, labels in train_gen:
#     print(frames[0].shape, labels[0])
    
#     ix += 1
#     if ix == 5:
#         break

In [0]:
NUM_FRAMES = 16
FRAME_HEIGHT = 224
FRAME_WIDTH = 224
NUM_RGB_CHANNELS = 3
NUM_CLASSES = 2

In [0]:
rgb_model = Inception_Inflated3d(include_top=False, weights='rgb_kinetics_only', 
                                 input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS), 
                                 classes=NUM_CLASSES)

In [0]:
x = rgb_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=rgb_model.input, outputs=output)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_gen, steps_per_epoch=5, epochs=100)